# deepsort

In [ ]:
import sys
sys.path.append('/kaggle/input/deepsort/deep_sort_pytorch/')
from deep_sort.deep_sort import DeepSort
from utils.parser import get_config

In [ ]:
# params can change

# data file
#mapping_result_df_path = '/kaggle/working/test_mapping.csv'
# mapping_result_df_path = '/kaggle/input/predeepsort/test_mapping_v2.csv'
# deep sort

REID_CKPT= "/kaggle/input/deepsort-data/ckpt.t7"

# in cascade match, if larger than this dist, just ignore that match
MAX_DIST= 0.2

# detection conf should be larger than this value
MIN_CONFIDENCE= 0.3

# value of NMS, = 1 means do nothing
NMS_MAX_OVERLAP= 1

# max iou, used in iou match
MAX_IOU_DISTANCE= 1

# how many times not updated, track will be delete, 
# life last, for how many frames
MAX_AGE= 30

# track to confirmed, you need to have how many hits
N_INIT= 1

# for each class, largest possible number
NN_BUDGET= 22

In [ ]:
def deepsort_helmets(video_name,
                     video_data,
                     video_dir,
                     plot=False,
                     plot_frames=[]):
    
    # Setup Deepsort 
    deepsort = DeepSort(model_path=REID_CKPT,
                        max_dist=MAX_DIST,
                        min_confidence=MIN_CONFIDENCE,
                        #nms_max_overlap=NMS_MAX_OVERLAP,
                        max_iou_distance=MAX_IOU_DISTANCE,
                        max_age=MAX_AGE,
                        n_init=N_INIT,
                        nn_budget=NN_BUDGET,
                        use_cuda=True)
    
    # Run through frames.
    video_data = video_data.sort_values('frame').reset_index(drop=True)
    one_video_result = []
    
    for frame, frame_data in tqdm(video_data.groupby(['frame']), total=video_data['frame'].nunique()):
        frame_data['x'] = (frame_data['left'] + round(frame_data['width'] / 2))
        frame_data['y'] = (frame_data['top'] + round(frame_data['height'] / 2))
        #二维列表
        xywhs = frame_data[['x','y','width','height']].values
        
        #打开视频文件
        cap = cv2.VideoCapture(f'{video_dir}/{video_name}.mp4')
        # frame-1 is designed by cv2 library, means read no.of frame = frame
        #set一些属性 （下一个要解码/捕获的帧基于0索引）
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1)
        success, image = cap.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 不筛选conf 其实mapping的时候已经筛选过一遍了
        confs = np.ones([len(frame_data),])
        # 全部都是helment 不分class
        clss =  np.zeros([len(frame_data),])
        
        one_frame_outputs = deepsort.update(xywhs, confs, clss, image)
        one_frame_deepsort_result = pd.DataFrame(one_frame_outputs, columns=['left','top','right','bottom','track_id','class_id'])
        
        if len(one_frame_deepsort_result) > 0:
            # TODO Fix this messy merge
            # deepsort的结果 和 detection的结果对应起来，即赋予每个bbox一个traking id
            #sort_values按多列排序，第一列排好后（内部）第二列也要排
            frame_data = pd.merge_asof(frame_data.sort_values(['left','top']),
                                       one_frame_deepsort_result[['left','top','track_id']].sort_values(['left','top']), 
                                       on='left', 
                                       suffixes=('','_deepsort'), #  相同列名的话加后缀
                                       direction='nearest')
            
        one_video_result.append(frame_data)
    #concat沿特定轴连接pandas对象 （串联起来）
    one_video_result = pd.concat(one_video_result)
    
    return one_video_result

In [ ]:
def add_deepsort_label_col(one_video_result):
    # col has:
    # video_frame, left, width, top, height, label
    # video, frame, x, y, top_deepsort, track_id
    
    # one_video_result is the return value of deepsort_helments
    
    # Find the top occuring label for each deepsort_cluster
    # value_counts: Return a Series containing counts of unique values.
    # to_frame: convert series to dataframe
    # reset_index: Reset the index of the DataFrame, and use the default one instead.
    # to_dict: Convert the DataFrame to a dictionary.
#     sortlabel_map = one_video_result.groupby('track_id')['label'].value_counts() \
#         .sort_values(ascending=False).to_frame() \
#         .rename(columns={'label':'label_count'}) \
#         .reset_index() \
#         .groupby(['track_id']) \
#         .first()['label'].to_dict()
    
    # Find the # of times that label appears for the deepsort_cluster.
#     sortlabelcount_map = one_video_result.groupby('track_id')['label'].value_counts() \
#         .sort_values(ascending=False).to_frame() \
#         .rename(columns={'label':'label_count'}) \
#         .reset_index() \
#         .groupby(['track_id']) \
#         .first()['label_count'].to_dict()
    
    #我写的：不考虑前后帧，只针对某个人track_id(deepsort_cluster)进行分析
    #选择len_diff最小的label作为可信的label(true_label)
    #[one_video_result["len_diff"]<1]
    truelabel_map = one_video_result[one_video_result["len_diff"]<1].groupby('track_id')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['track_id']) \
        .first()['label'].to_dict()
    
    one_video_result['label_deepsort'] = one_video_result['track_id'].map(truelabel_map)
#     one_video_result['label_deepsort'] = one_video_result['track_id'].map(sortlabel_map)
#     one_video_result['label_count_deepsort'] = one_video_result['track_id'].map(sortlabelcount_map)

    return one_video_result

In [ ]:
# Add video and frame columns to submission.
mapping_result_df['video'] = mapping_result_df['video_frame'].str.split('_').str[:3].str.join('_')
mapping_result_df['frame'] = mapping_result_df['video_frame'].str.split('_').str[-1].astype('int')

if debug:
    video_dir = '../input/nfl-health-and-safety-helmet-assignment/train/'
else:
    video_dir = '../input/nfl-health-and-safety-helmet-assignment/test/'

# Loop through test videos and apply. If in debug mode show the score change.
all_videos_result = []
all_videos_change_label_result = []
count = 0
for video_name, video_data in tqdm(mapping_result_df.groupby('video'), total=mapping_result_df['video'].nunique()):
    print(f'==== {video_name} ====')
    if debug:
        # Plot deepsort labels when in debug mode.
        one_video_result = deepsort_helmets(video_name, video_data, video_dir, plot_frames=[10, 150, 250])
    else:
        one_video_result = deepsort_helmets(video_name, video_data, video_dir)
    #one_video_result.groupby('track_id').apply(lambda x: x.sort_values('len_diff', ascending=False))
    all_videos_result.append(one_video_result)
    one_video_change_label_result = add_deepsort_label_col(one_video_result)
    all_videos_change_label_result.append(one_video_change_label_result)
#     if debug:
#         # Score
#         score_vs_deepsort(myvideo, out, labels)
        
mapping_deepsort_result = pd.concat(all_videos_change_label_result).copy()

### rename submission df

In [ ]:
ss = pd.read_csv('../input/nfl-health-and-safety-helmet-assignment/sample_submission.csv')
# Final Checks
mapping_deepsort_result['label_deepsort'] = mapping_deepsort_result['label_deepsort'] \
    .fillna(mapping_deepsort_result['label'])
mapping_deepsort_result = mapping_deepsort_result.drop('label', axis=1) \
    .rename(columns={'label_deepsort':'label'})[ss.columns]

In [ ]:
# Drop duplicate labels
mapping_deepsort_result = mapping_deepsort_result.drop_duplicates(subset=["video_frame", "label"], keep='first')

In [ ]:
mapping_deepsort_result

# save final result

In [ ]:
submission_df_path = '/kaggle/working/submission.csv'
mapping_deepsort_result.to_csv(submission_df_path, index=False)